<h1>Infaltion Analysis</h1>

In [517]:
from fredapi import Fred
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [518]:
#My key to obtain the api
super_secret_key = '9ab9ae52f27e006f9acc6d210f76105a'

#Pulling from FRED
fred = Fred(api_key = super_secret_key)

In [519]:
#List of the Series ID

series_data = {
"CPI" : "CPIAUCSL",  #CPI US City Average
"Core_CPI" : "CPILFESL", #Core CPI US City Average
"PCE" : "PCE",
"Core_PCE" : "PCEPILFE",
"Federal_Funds_Rate" : "FEDFUNDS"

}

for series_name, series_id in series_data.items():
  series_data[series_name] = fred.get_series(series_id = series_id, observation_start = '01/01/2006', observation_end = '09/01/2024')

In [520]:
df = pd.DataFrame.from_dict(series_data)

In [521]:
col_list = ['CPI', 'Core_CPI', 'PCE', 'Core_PCE']

for col_name in col_list:
    # Monthly Change
    col_name_month = col_name + " Month %"
    df[col_name_month] = round(df[col_name].pct_change() * 100,2)

    # 3 Month Change
    col_name_3month = col_name + " 3 Month %"
    df[col_name_3month] = round(df[col_name].pct_change(periods = 3) * 100,2)

    # 6 Month Change
    col_name_6month = col_name + " 6 Month %"
    df[col_name_6month] = round(df[col_name].pct_change(periods = 6) * 100,2)

    # 9 Month Change
    col_name_9month = col_name + " 9 Month %"
    df[col_name_9month] = round(df[col_name].pct_change(periods = 9) * 100,2)

    # 1 Year Change
    col_name_year = col_name + " Year %"
    df[col_name_year] = round(df[col_name].pct_change(periods = 12) * 100,2)

In [522]:
# Making Edit
df.reset_index(inplace = True)
df.rename(columns = {'index': 'Date'}, inplace = True)

df['Date'] = pd.to_datetime(df['Date'])

df = df[df['Date'] >= '2016-01-01']

In [523]:
df.columns

Index(['Date', 'CPI', 'Core_CPI', 'PCE', 'Core_PCE', 'Federal_Funds_Rate',
       'CPI Month %', 'CPI 3 Month %', 'CPI 6 Month %', 'CPI 9 Month %',
       'CPI Year %', 'Core_CPI Month %', 'Core_CPI 3 Month %',
       'Core_CPI 6 Month %', 'Core_CPI 9 Month %', 'Core_CPI Year %',
       'PCE Month %', 'PCE 3 Month %', 'PCE 6 Month %', 'PCE 9 Month %',
       'PCE Year %', 'Core_PCE Month %', 'Core_PCE 3 Month %',
       'Core_PCE 6 Month %', 'Core_PCE 9 Month %', 'Core_PCE Year %'],
      dtype='object')

In [524]:
# Creating a list
CPI = df[['Date' ,'CPI', 'CPI Month %', 'CPI 3 Month %', 'CPI 6 Month %', 'CPI 9 Month %', 'CPI Year %']]

In [525]:
CPI.columns

Index(['Date', 'CPI', 'CPI Month %', 'CPI 3 Month %', 'CPI 6 Month %',
       'CPI 9 Month %', 'CPI Year %'],
      dtype='object')

In [561]:
# import packages
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px

# start the app
app = Dash()

# App layout
app.layout = [
    # First Line
    html.Div(children = 'Inflation Over the Years', style = {'fontSize' : 50, 'color': '#202A44', 'textAlign': 'center'}),
    html.Div(children = 'BY: David Sierra Perez' , style = {'fontSize' : 20, 'color': '#202A44', 'textAlign': 'center'}),

    # Creates a Line
    html.Hr(),

    # Add the Data Table
    dash_table.DataTable(data = CPI.to_dict('records'), page_size = 5),

    # Adding a Graph
    dcc.Graph(figure = px.line(CPI, x = 'Date', y = 'CPI Year %')),
]

# Run the app
if __name__ == '__main__':
    app.run(debug = True)

In [580]:
# Predefind Classes

colors = {
    'background' : 'white',
    'title_text' : '#202A44',
    'graph_text' : 'black',
    'title_size' : 50,
    'other_text' : 20
}

In [583]:
# import packages
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px

# start the app
app = Dash()

# App layout
app.layout = [
    html.Div([
    
    # First Line
    html.Div(children = 'Inflation Over the Years', style = {'fontSize' : colors['title_size'], 'color': colors['title_text'], 'textAlign': 'center'}),
    html.Div(children = 'BY: David Sierra Perez' , style = {'fontSize' : colors['other_text'], 'color': colors['title_text'], 'textAlign': 'center'}),

    # Creates a Line
    html.Hr(),

    # Adding Radio Buttons
    html.Div(
    dcc.RadioItems(options = ['CPI Month %', 'CPI 3 Month %', 'CPI 6 Month %', 'CPI 9 Month %', 'CPI Year %'], value = 'CPI Year %', id = 'controls-and-radio-item', inline = True),
    style = {'textAlign' : 'center'}),

    # Adding another Line
    html.Hr(),

    # Creating a Space
    html.Br(),

    html.Div([
        # Adding the interactive graph figure
        dcc.Graph(figure = {}, id = 'controls-and-graph')
        ], style = {'width' : "100%", 'height': '100%'})
    


    ])
    ]
# Add controls to build the interaction
@callback(
    Output(component_id = 'controls-and-graph', component_property = 'figure'),
    Input(component_id = 'controls-and-radio-item', component_property = 'value')
)

def update_graph(col_chosen):
    fig = px.line(CPI, x = 'Date', y = col_chosen, template = 'simple_white')
    return fig

# Run the app
if __name__ == '__main__':
    app.run(debug = True)